In [20]:
import pickle
import os
import json
from collections import Counter

import sys
sys.path.append("../../Interactive_Graph_Visualizer/networkx-master")
import networkx as nx

In [21]:
def domain_detect(url):
    FQDN=url.split("/")[2]
    FQDN_list=FQDN.split(".")
    return u".".join(FQDN_list[-3:])#開始点がリスト長より長くても問題なく無く動く

In [22]:
search_word = u"iPhone"
max_page = 10
root_dir = "../Data/Search_" + search_word + "_" + unicode(max_page) + "_add_childs_append"
src_pages_dir = os.path.join(root_dir,"Pages")
text_file = "../Data/Search_iPhone_10_add_childs_append/K10_myexttext_largest/nx_datas/G_with_params_comp4_2_hits_hosts.txt"
G_file = "../Data/Search_iPhone_10_add_childs_append/K10_myexttext_largest/nx_datas/G_with_params_comp4_2.gpkl"
with open(G_file,'r') as fi:
    G = pickle.load(fi)

In [23]:
node_datas = dict()
for node in G:
    with open(os.path.join(src_pages_dir,str(node)+".json"),"r") as fj:
        node_data = json.load(fj)
    node_datas[node] = node_data

In [24]:
len(G)

218

In [25]:
G.node[2]

{'a_score': 0.00017401832689457823,
 'color': '#EB6100',
 'h_score': 6.79129919615872e-08,
 'topic': 0}

In [26]:
for node in G:
    node_data = node_datas[node]
    from_hosts = []
    to_hosts = []
    for in_edge in G.in_edges(node):#入ってくるエッジ
        from_node = in_edge[0]
        fromnode_data = node_datas[from_node]
        from_hosts.append(domain_detect(fromnode_data["url"]))
    for out_edge in G.out_edges(node):#出て行くエッジ
        to_node = out_edge[1]
        tonode_data = node_datas[to_node]
        to_hosts.append(domain_detect(tonode_data["url"]))

    G.node[node]["from_hosts"] = dict(Counter(from_hosts))#Counterインスタンスを辞書型でキャスト
    G.node[node]["to_hosts"] = dict(Counter(to_hosts))

In [27]:
G.node[2]

{'a_score': 0.00017401832689457823,
 'color': '#EB6100',
 'from_hosts': {u'time-space.kddi.com': 3, u'tobuy.jp': 1, u'www.kddi.com': 1},
 'h_score': 6.79129919615872e-08,
 'to_hosts': {u'onlineshop.au.com': 1,
  u'star.auone.jp': 1,
  u'time-space.kddi.com': 5,
  u'wowma.jp': 1,
  u'www.kddi.com': 7},
 'topic': 0}

In [29]:
with open(G_file,"w") as fo:
    pickle.dump(G,fo)
with open(text_file,"w") as fo:
    print >> fo,"この時点で、ノードには代表トピックとその色とオーソリティ・ハブスコア、from_hosts、to_hostsをエッジには重みの情報を渡している。"
    print >> fo,"from_hostはリンクされているWebページのドメインとその件数、to_hostsはリンクしているWebページのドメインとその件数。"
    print >> fo,"ノード数：" + str(len(G.node.keys())) + "（変化なし）"